# SQL Subqueries

## Introduction

SQL queries can get complex. For example, you might have been a little thrown off by the many to many join in the last lab. There, you had to join four tables. This is just the tip of the iceberg. Depending on how your database is set up, you might have to join subset views of multiple tables. When queries get complex like this, it is often useful to use the concept of subqueries to help break the problem into smaller, more digestible tasks.

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## Our Customer Relationship Management ERD

As a handy reference, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('data.sqlite')

## Substituting `JOIN` with Subqueries

Let's start with a query of employees from the United States. Using your current knowledge, you could solve this using a join.

In [3]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
JOIN offices
    USING(officeCode)
WHERE country = "USA"
;"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Bow,Anthony,1
1,Firrelli,Jeff,1
2,Jennings,Leslie,1
3,Murphy,Diane,1
4,Patterson,Mary,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


Another approach would be to use a subquery. Here's what it would look like:

In [4]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
WHERE officeCode IN (SELECT officeCode
                     FROM offices 
                     WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bow,Anthony,1
4,Jennings,Leslie,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


In [14]:
# Find customers and their total order amounts and use sub queries

q = """
SELECT customerNumber, customerName, SUM(amount) AS total
FROM customers
JOIN payments
    USING(customerNumber)
GROUP BY customerNumber
;
"""
pd.read_sql(q, conn)

,customerNumber,customerName,total
0,103,Atelier graphique,22314.36
1,112,Signal Gift Stores,80180.98
2,114,"Australian Collectors, Co.",180585.07
3,119,La Rochelle Gifts,116949.68
4,121,Baane Mini Imports,104224.79
...,...,...,...
93,486,Motor Mint Distributors Inc.,77726.59
94,487,Signal Collectibles Ltd.,42570.37
95,489,"Double Decker Gift Stores, Ltd",29586.15
96,495,Diecast Collectables,65541.74


There it is, a query within a query! This can be very helpful and also allow you to break down problems into constituent parts. Often queries can be formulated in multiple ways as with the above example. Other times, using a subquery might be essential. For example, what if you wanted to find all of the employees from offices with at least 5 employees?

## Subqueries for Filtering Based on an Aggregation

Think for a minute about how you might write such a query.  

Now that you've had a minute to think it over, you might see some of the challenges with this query. On the one hand, we are looking to filter based on an aggregate condition: the number of employees per office. You know how to do this using the `GROUP BY` and `HAVING` clauses, but the data we wish to retrieve is not aggregate data. We only wish to **filter** based on the aggregate, not retrieve aggregate data. As such, this is a natural place to use a subquery.

In [22]:
# customers with the highest amount of orders
q = """
SELECT customerName, totalOrders
FROM (
    SELECT customers.customerName, 
           COUNT(orders.orderNumber) AS totalOrders,
           customers.customerNumber
      FROM customers
      JOIN orders ON customers.customerNumber = orders.customerNumber
     GROUP BY customers.customerNumber, customers.customerName
) AS customer_order_counts
ORDER BY totalOrders DESC;
"""

pd.read_sql(q, conn)

,customerName,totalOrders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17
2,"Australian Collectors, Co.",5
3,Danish Wholesale Imports,5
4,"Dragon Souveniers, Ltd.",5
...,...,...
93,"Iberia Gift Imports, Corp.",2
94,Signal Collectibles Ltd.,2
95,"Double Decker Gift Stores, Ltd",2
96,Diecast Collectables,2


In [29]:
# find the most expensive product ordered by each customer

q = """
SELECT customerNumber, customerName, productName, MAX(amount) AS total
FROM customers
JOIN payments
    USING(customerNumber)
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products
    USING(productCode)
GROUP BY customerNumber
;"""
pd.read_sql(q, conn)

,customerNumber,customerName,productName,total
0,103,Atelier graphique,1965 Aston Martin DB5,14571.44
1,112,Signal Gift Stores,1917 Grand Touring Sedan,33347.88
2,114,"Australian Collectors, Co.",1996 Moto Guzzi 1100i,82261.22
3,119,La Rochelle Gifts,1969 Harley Davidson Ultimate Chopper,49523.67
4,121,Baane Mini Imports,1952 Alpine Renault 1300,50218.95
...,...,...,...,...
93,486,Motor Mint Distributors Inc.,1993 Mazda RX-7,45994.07
94,487,Signal Collectibles Ltd.,1937 Lincoln Berline,29997.09
95,489,"Double Decker Gift Stores, Ltd",1972 Alfa Romeo GTA,22275.73
96,495,Diecast Collectables,1962 LanciaA Delta 16V,59265.14


In [25]:
# Find products that have never been ordered
q = """
SELECT productCode, productName
FROM products
WHERE productCode NOT IN (
    SELECT DISTINCT productCode
    FROM orderdetails
)
;"""
pd.read_sql(q, conn)

,productCode,productName
0,S18_3233,1985 Toyota Supra


In [5]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
WHERE officeCode IN (
    SELECT officeCode 
    FROM offices 
    JOIN employees
        USING(officeCode)
    GROUP BY 1
    HAVING COUNT(employeeNumber) >= 5
)
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bondur,Gerard,4
4,Bow,Anthony,1
5,Jennings,Leslie,1
6,Thompson,Leslie,1
7,Bondur,Loui,4
8,Hernandez,Gerard,4
9,Castillo,Pamela,4


You can chain queries like this in many fashions. For example, maybe you want to find the average of individual customers' average payments:

(It might be more interesting to investigate the standard deviation of customer's average payments, but standard deviation is not natively supported in SQLite as it is in other SQL versions like PostgreSQL.)

In [6]:
q = """
SELECT AVG(customerAvgPayment) AS averagePayment
FROM (
    SELECT AVG(amount) AS customerAvgPayment
    FROM payments
    JOIN customers
        USING(customerNumber)
    GROUP BY customerNumber
)
;"""
pd.read_sql(q, conn)

,averagePayment
0,31489.754582


You can also run subqueries that reference keys with different names between different tables. For example you can use the employee number in the employees table and the matching sales rep employee number in the customers table.

In [7]:
q = """
SELECT lastName, firstName, employeeNumber
FROM employees
WHERE employeeNumber IN (SELECT salesRepEmployeeNumber
                     FROM customers 
                     WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,employeeNumber
0,Jennings,Leslie,1165
1,Thompson,Leslie,1166
2,Firrelli,Julie,1188
3,Patterson,Steve,1216
4,Tseng,Foon Yue,1286
5,Vanauf,George,1323


## Summary

In this lesson, you were briefly introduced to the powerful concept of subqueries and how you can use them to write more complex queries. In the upcoming lab, you'll really start to strengthen your SQL and data wrangling skills by using all of the SQL techniques introduced thus far.